In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import torch
import torch.nn as nn


from pathlib import Path
import os
from classes import *

In [56]:
loaddata = LoadData()


In [57]:

file = str(Path('FA_20240517_2H_yeast_Nicotinamide-d4 _9.csv'))
substrates = loaddata.get_substrate_list(file)
metabolites = loaddata.get_metabolite_list(file)
df_list = loaddata.load_data(file)

In [58]:
substrates, metabolites

([9.094, 8.876, 8.42, 7.772, np.float64(4.7)], [9.031, 8.714, 8.376, 7.659])

In [78]:
from scipy.ndimage import gaussian_filter1d


def fill_df(df, column):
    """If Data is not ranging from -2 to 10, fill the data with noise
    
    """
    # renmame the columns
    df.columns = ['x', column]
    x = df.loc[:, 'x']
    y = df.loc[:, column]



    # Calculate the step size of the x values
    x_diff = np.diff(x)
    step = np.mean(x_diff)
    
    # Take sample range for the noise
    x_range_lower = 20
    x_range_upper = 100

    # get sample data of the noise
    x_sample = x[x_range_lower:x_range_upper]
    y_sample = y[x_range_lower:x_range_upper]

    # get the max and min values of the sample data
    y_min, y_max = y_sample.min(), abs(y_sample.min())

    # get the min max values of the x data
    x_min, x_max = x.min(), x.max()

    if x_min > -2:

        # create values in n steps
        x_new = np.arange(-2, x.iloc[0], step)
        y_new = np.zeros_like(x_new)

        # set the noise level
        noise = np.random.normal(y_min, y_max, len(x_new))
        
        # smooth the noise with gaussian filter
        # Berechne die Standardabweichung der y-Daten
        sigma = len(y_new) / 100
        noise = gaussian_filter1d(input=noise, sigma=sigma)

        # replace y_new with noise
        y_new = noise

        data = pd.DataFrame({'x': x_new, column: y_new})
        df = pd.concat([data, df], axis=0)
        df.reset_index(drop=True, inplace=True)

    if x_max < 13:
    
        # create values in n steps
        x_new = np.arange(x.iloc[-1], 13, step)
        #print(x_new)
        y_new = np.zeros_like(x_new)

        # set the noise level
        noise = np.random.normal(y_min, y_max, len(x_new))

        # smooth the noise with gaussian filter
        # Berechne die Standardabweichung der y-Daten
        sigma = len(y_new) / 100
        noise = gaussian_filter1d(input=noise, sigma=sigma)

        # replace y_new with noise
        y_new = noise

        data = pd.DataFrame({'x': x_new, column: y_new})
        df = pd.concat([df, data], axis=0)
        df.reset_index(drop=True, inplace=True)
    #display(df)
    return df

Expand the Dataframe from -2 to 10.

In [79]:
df = pd.read_csv(df_list[0])

df_merged = pd.DataFrame()
# Iterate over each column starting from the second column
for i in range(1, len(df.columns)):
    #print(df.columns[i], i)
    # Pass a subset to fill_df, without modifying df in the loop
    filled_df = fill_df(df.iloc[:, [0, i]], column=df.columns[i])
    df_merged['x'] = filled_df['x']
    #display(filled_df)
    # Process filled_df if needed (e.g., store it in a list or combine with df)
    df_merged = pd.merge(df_merged, filled_df[df.columns[i]], left_index=True, right_index=True, how='outer')
display(df_merged)
print(df_merged.shape)

,x,FA_20240517_2H_yeast_1.9.ser#1,FA_20240517_2H_yeast_1.9.ser#2,FA_20240517_2H_yeast_1.9.ser#3,FA_20240517_2H_yeast_1.9.ser#4,FA_20240517_2H_yeast_1.9.ser#5,FA_20240517_2H_yeast_1.9.ser#6,FA_20240517_2H_yeast_1.9.ser#7,FA_20240517_2H_yeast_1.9.ser#8,FA_20240517_2H_yeast_1.9.ser#9,...,FA_20240517_2H_yeast_1.9.ser#41,FA_20240517_2H_yeast_1.9.ser#42,FA_20240517_2H_yeast_1.9.ser#43,FA_20240517_2H_yeast_1.9.ser#44,FA_20240517_2H_yeast_1.9.ser#45,FA_20240517_2H_yeast_1.9.ser#46,FA_20240517_2H_yeast_1.9.ser#47,FA_20240517_2H_yeast_1.9.ser#48,FA_20240517_2H_yeast_1.9.ser#49,FA_20240517_2H_yeast_1.9.ser#50
0,-2.000000,-3293.274866,-2771.350384,-929.741544,-1359.283321,-1328.503946,-2922.599020,-2643.083209,-807.724927,-1651.961061,...,-3295.348110,-1250.484380,-2115.195004,-2103.049343,-1785.815304,-2367.302409,-2748.903517,-874.394683,-1960.532828,-3068.348436
1,-1.991855,-3244.393768,-2779.388580,-959.673299,-1356.923988,-1323.744349,-2918.229794,-2647.148154,-807.486051,-1638.742847,...,-3247.070285,-1234.837146,-2113.190155,-2121.284450,-1780.041450,-2350.402046,-2737.561299,-891.850457,-1942.533329,-3068.183938
2,-1.983710,-3154.835188,-2795.123351,-1017.450181,-1352.114536,-1314.001604,-2909.283176,-2652.805305,-807.442506,-1614.266326,...,-3155.158629,-1204.612169,-2109.738416,-2157.003679,-1768.532506,-2316.381207,-2716.099339,-925.893516,-1908.174501,-3067.355971
3,-1.975565,-3039.210166,-2817.821912,-1099.967570,-1344.759995,-1298.959926,-2895.405156,-2656.131691,-808.025357,-1582.307004,...,-3028.413898,-1161.942026,-2105.968825,-2208.373920,-1751.538541,-2264.640971,-2686.901695,-975.282317,-1860.739293,-3065.375839
4,-1.967420,-2916.933833,-2846.195240,-1202.860781,-1334.818053,-1278.384642,-2876.062179,-2651.923394,-810.082704,-1547.722175,...,-2878.405105,-1110.192202,-2103.225558,-2272.794259,-1730.184321,-2195.123109,-2653.252729,-1037.962511,-1804.216835,-3061.460981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,12.967206,-3237.123798,-2609.612829,-649.453985,-1677.959529,-1094.781806,-978.173392,-3154.248903,-685.956524,-650.220549,...,-2098.100763,-2069.969897,-688.557611,-1012.412275,-1959.984261,-2927.759210,-1735.089854,-1042.706745,-1927.175243,-3928.856268
1840,12.975351,-3349.850877,-2799.825285,-825.176796,-1682.963561,-959.907999,-1323.292834,-2438.880938,-397.246982,-754.261521,...,-2177.338274,-2032.645395,-629.006219,-1037.547403,-2324.155823,-3296.407613,-1335.353266,-1013.633182,-1748.735288,-4158.649547
1841,12.983496,-3376.000509,-3148.321698,-946.159874,-1701.882023,-833.917884,-2077.958982,-1894.441798,-148.194181,-888.122935,...,-2216.525322,-1902.897614,-650.269625,-1308.956327,-2888.000872,-3637.610838,-1044.220055,-1019.220750,-1617.147992,-4266.260923
1842,12.991641,-3310.639537,-3515.337008,-960.599020,-1727.800097,-740.369503,-2888.148969,-1610.440540,3.760232,-1014.824286,...,-2231.307272,-1761.700572,-738.733085,-1714.128369,-3441.378351,-3925.362125,-872.571684,-1042.534908,-1590.304910,-4264.247138


(1844, 51)


In [80]:
def get_pictrue(df):
    pic = df.iloc[:, 1:]
    return pic

pic = get_pictrue(df)

print(pic.shape)

x = df.iloc[:, 0]
print(x.shape)

(1063, 50)
(1063,)


In [82]:
import numpy as np
from scipy.ndimage import zoom

# Original array
# Target shape
target_shape = (500, 50)


# Calculate scaling factors for each dimension
scale_factors = (target_shape[0] / pic.shape[0], target_shape[1] / pic.shape[1])

# Apply zoom with the calculated scale factors
upscaled_array = zoom(pic, scale_factors, order=3)


x_shape = (pic.shape[0], )
scale_factors_x = (target_shape[0] / x_shape[0], )
upscaled_x = zoom(x, scale_factors_x, order=3)


print(upscaled_array.shape)
print(upscaled_x.shape)


(500, 50)
(500,)


In [83]:
import plotly.express as px
y_labels = upscaled_x

def get_x_labels(column_list):
    x_labels = []
    for i in column_list:

        x_labels.append(i.split('#')[-1])
    return x_labels

x_labels = get_x_labels(df.columns[1:])

fig = px.imshow(upscaled_array, color_continuous_scale='Spectral_r', aspect='auto', 
                #y=y_labels, x=x_labels,
                )
fig.show()

In [64]:
print(upscaled_x)

[ 2.52528     2.54255518  2.55994198  2.57728178  2.59461597  2.61194971
  2.62928357  2.64661769  2.6639522   2.68128716  2.69862247  2.71595798
  2.73329358  2.75062915  2.76796455  2.78529966  2.80263435  2.81996861
  2.83730255  2.85463631  2.87197     2.88930376  2.9066377   2.92397197
  2.94130667  2.95864178  2.97597719  2.99331275  3.01064836  3.02798387
  3.04531917  3.06265412  3.07998863  3.09732274  3.11465659  3.13199031
  3.14932401  3.16665786  3.18399212  3.20133632  3.21867072  3.23600479
  3.25333861  3.27067231  3.28800602  3.30533988  3.32267401  3.34000853
  3.35734349  3.3746788   3.39201432  3.40934992  3.42668549  3.44402088
  3.46135598  3.47869067  3.49602492  3.51335886  3.53069262  3.54802631
  3.56536007  3.58269402  3.60002829  3.617363    3.63469812  3.65203352
  3.66936909  3.68670469  3.7040402   3.7213755   3.73871044  3.75604494
  3.77337905  3.7907129   3.80804662  3.82538032  3.84271414  3.86004821
  3.87738265  3.89471753  3.91205279  3.92938827  3